# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#store file path
weather_file = "../WeatherPy/output.csv"
#read file
weather_df = pd.read_csv(weather_file, encoding="ISO-8859-1")
#show the header
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,east london,-33.0153,27.9116,64.40,82,75,11.50,ZA,1618914033
1,acarau,-2.8856,-40.1200,75.34,91,94,4.76,BR,1618914034
2,antofagasta,-23.6500,-70.4000,57.20,82,0,2.30,CL,1618914034
3,rokytne,51.2796,27.2140,50.18,65,100,7.83,UA,1618914034
4,sept-iles,50.2001,-66.3821,39.20,93,100,4.61,CA,1618914035


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)

#use lat and lng and humidity
locations = weather_df[["Lat", "Lng"]]

humidity = weather_df["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=20,
                                 point_radius=1)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#Narrow down the cities to cities that fill ideal conditions
ideal_weather = weather_df[(70 < weather_df['Max Temp'])
                                           &(weather_df['Max Temp'] <80) 
                                           & (weather_df['Wind Speed'] < 10) 
                                           & (weather_df['Cloudiness'] == 0)]
ideal_weather


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
75,hohhot,40.8106,111.6522,75.20,11,0,6.71,CN,1618914067
88,cape town,-33.9258,18.4232,75.20,46,0,9.22,ZA,1618914072
120,fushun,41.8558,123.9233,75.20,14,0,2.24,CN,1618914086
243,maceio,-9.6658,-35.7353,77.00,94,0,3.44,BR,1618913985
314,mogwase,-25.1850,27.2665,72.68,30,0,8.97,ZA,1618914164
323,thabazimbi,-24.5917,27.4116,79.05,24,0,6.26,ZA,1618914168
324,passo de camaragibe,-9.2368,-35.4901,77.00,94,0,3.44,BR,1618914168
362,fochville,-26.4886,27.4939,71.01,50,0,1.01,ZA,1618914181


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
#copy ideal weather data to add new file and rename with new variable
hotel_df = ideal_weather.copy()
hotel_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
75,hohhot,40.8106,111.6522,75.20,11,0,6.71,CN,1618914067
88,cape town,-33.9258,18.4232,75.20,46,0,9.22,ZA,1618914072
120,fushun,41.8558,123.9233,75.20,14,0,2.24,CN,1618914086
243,maceio,-9.6658,-35.7353,77.00,94,0,3.44,BR,1618913985
314,mogwase,-25.1850,27.2665,72.68,30,0,8.97,ZA,1618914164
323,thabazimbi,-24.5917,27.4116,79.05,24,0,6.26,ZA,1618914168
324,passo de camaragibe,-9.2368,-35.4901,77.00,94,0,3.44,BR,1618914168
362,fochville,-26.4886,27.4939,71.01,50,0,1.01,ZA,1618914181


In [6]:
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
75,hohhot,40.8106,111.6522,75.20,11,0,6.71,CN,1618914067,
88,cape town,-33.9258,18.4232,75.20,46,0,9.22,ZA,1618914072,
120,fushun,41.8558,123.9233,75.20,14,0,2.24,CN,1618914086,
243,maceio,-9.6658,-35.7353,77.00,94,0,3.44,BR,1618913985,
314,mogwase,-25.1850,27.2665,72.68,30,0,8.97,ZA,1618914164,
323,thabazimbi,-24.5917,27.4116,79.05,24,0,6.26,ZA,1618914168,
324,passo de camaragibe,-9.2368,-35.4901,77.00,94,0,3.44,BR,1618914168,
362,fochville,-26.4886,27.4939,71.01,50,0,1.01,ZA,1618914181,


In [7]:
#loop through the new dataframe to find and add hotel name

hotel_names = []
for index, row in hotel_df.iterrows():    
   
    radius = 5000
    target_type = "lodging"
     
    lat = row["Lat"]
    lng = row["Lng"]
    
    location = f"{lat},{lng}"
    
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    params = {"location": location,
              "radius": radius,  
              "type": target_type, 
              "key": g_key}
    
    
    try:
        response = requests.get(base_url, params=params)
        hotel_name = response.json()
        hotel_names.append(hotel_name["results"][0]["name"])
        print(hotel_name["results"][0]["name"]) 
    
    except:
        hotel_names.append("None")
        print("None found. Skipping ...")
        pass


Sheraton Hohhot Hotel
Southern Sun Waterfront Cape Town
Love Theme Apartment Xinhua Branch
Poutur Pousada
De Skaap Accommodation & Event Venue
Eagle's Rest
Chácara vovó candinha
Eden Vreugd Guesthouse


In [8]:
#add the hotel names to the dataframe.
hotel_df["Hotel Name"] = hotel_names
hotel_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
75,hohhot,40.8106,111.6522,75.20,11,0,6.71,CN,1618914067,Sheraton Hohhot Hotel
88,cape town,-33.9258,18.4232,75.20,46,0,9.22,ZA,1618914072,Southern Sun Waterfront Cape Town
120,fushun,41.8558,123.9233,75.20,14,0,2.24,CN,1618914086,Love Theme Apartment Xinhua Branch
243,maceio,-9.6658,-35.7353,77.00,94,0,3.44,BR,1618913985,Poutur Pousada
314,mogwase,-25.1850,27.2665,72.68,30,0,8.97,ZA,1618914164,De Skaap Accommodation & Event Venue
323,thabazimbi,-24.5917,27.4116,79.05,24,0,6.26,ZA,1618914168,Eagle's Rest
324,passo de camaragibe,-9.2368,-35.4901,77.00,94,0,3.44,BR,1618914168,Chácara vovó candinha
362,fochville,-26.4886,27.4939,71.01,50,0,1.01,ZA,1618914181,Eden Vreugd Guesthouse


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer ontop of heat map
max_temp = hotel_df["Max Temp"]

heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, max_intensity=20, dissipating=False, point_radius = 1)

heat_marker = gmaps.marker_layer(locations, info_box_content = hotel_info)

heatmap_fig.add_layer(heat_marker)

heatmap_fig.add_layer(heat_layer)

heatmap_fig

# Display figure


Figure(layout=FigureLayout(height='420px'))